# Geometry description with the level set function
We describe how to use a level set description in ngsxfem.

Let $\phi$ be a continuous level set function. 

We define the domains and the interface

$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

In [27]:
import netgen.gui
%gui tk
import tkinter

# the constant pi
from math import pi
# ngsolve stuff
from ngsolve import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *

We generate the background mesh of the domain and use a simplicial triangulation

In [28]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.5, quad_dominated=False))

bcs =  [1, 1, 1, 1]


On the background mesh we define the level set function. 

In the visualization we distinguish the negative and the positive part of the level set values corresponding to the subdomains that are described.

In [29]:
levelset = (sqrt(sqrt(x*x*x*x+y*y*y*y)) - 1.0)
Draw(levelset,mesh,"levelset")
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0


It is difficult to work with arbitrary level set function. To obtain something feasible, we interpolate into the space of piecewise linear functions. The thusly obtained approximation is straight within each element and hence is much easier to handle (e.g. for numerical integration).

In [30]:
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
Draw(lsetp1,mesh,"lsetp1")
visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

## Cut Information

In unfitted finite element it is important to know which elements are cut by the interface $ \{ \phi = 0 \} $. These informations are gathered in the cut information class:

In [31]:
ci = CutInfo(mesh, lsetp1)

### Marking elements

We can ask for BitArrays corresponding to the differently marked elements 
  * NEG : True if $\phi < 0$ everywhere on this element, else False
  * POS : True if $\phi > 0$ everywhere on this element, else False
  * IF : True if $\phi = 0$ somewhere on this element, else False  

In [32]:
ba_IF = ci.GetElementsOfType(IF)
cf_IF = BitArrayCF(ba_IF)
Draw(cf_IF,mesh,"cut_elements")

In [33]:
ba_NEG = ci.GetElementsOfType(NEG)
cf_NEG = BitArrayCF(ba_NEG)
Draw(cf_NEG,mesh,"neg_elements")

In [34]:
ba_POS = ci.GetElementsOfType(POS)
cf_POS = BitArrayCF(ba_POS)
Draw(cf_POS,mesh,"pos_elements")

Other collection of elements (which are uncut, which are not in NEG or POS) can also be easily obtained, e.g.

In [35]:
ba_uncut = BitArray(ba_NEG)
ba_uncut |= ba_POS
cf_uncut = BitArrayCF(ba_uncut)
Draw(cf_uncut,mesh,"uncut_elements")

In [36]:
ba_hasneg = BitArray(ba_NEG)
ba_hasneg |= ba_IF
cf_hasneg = BitArrayCF(ba_hasneg)
Draw(cf_hasneg,mesh,"hasneg_elements")

### Marking facets

Sometimes we also want to mark some facets depending on their neighbors, e.g. to add some stabilization terms there. By providing two element-based BitArray we obtain a BitArray for the facets where. Let A1,A2 be the BitArray-values for the left neighbor and B1,B2 be the BitArray-value for the right neighbor. There are two options to connect these BitArrays:
 * use_and=True (default): the result is True for a facet if (A1 and B2) or (B1 and A2)
 * use_and=False     (or): the result is True for a facet if (A1 or B2) or (B1 or A2)
with bnd_val_a and bnd_val_b (default: True) we can prescribe how the BitArray should be interpreted if there is only one neighbor (boundary).

#### Example 1:

We mark all facets that are on the outer boundary of the NEG-domain. We achieve this by connecting (with and) the IF-elements with the NEG-elements.

In [44]:
ba_facets = GetFacetsWithNeighborTypes(mesh,a=ba_NEG,b=ba_IF,bnd_val_a=False,bnd_val_b=False)

We can also go from facets to elements

In [45]:
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")

#### Example 2:

We mark all facets that have at least one neighbor in the NEG-domain.

In [62]:
ba_facets_NEG = GetFacetsWithNeighborTypes(mesh,a=ba_NEG,b=ba_NEG,bnd_val_a=False,
                                           bnd_val_b=False,use_and=False)
ba_extended_NEG = GetElementsWithNeighborFacets(mesh,ba_facets_NEG)
Draw(BitArrayCF(ba_extended_NEG), mesh, "extended_NEG")

We can successively apply this to extend from some part of the domain by direct neighbors, etc ....

In [67]:
ba_facets_NEG = GetFacetsWithNeighborTypes(mesh,a=ba_extended_NEG,b=ba_extended_NEG,
                                           bnd_val_a=False,bnd_val_b=False,use_and=False)
ba_extended_NEG = GetElementsWithNeighborFacets(mesh,ba_facets_NEG)
Draw(BitArrayCF(ba_extended_NEG), mesh, "extended_NEG")

### Marking dofs

Assume you want to solve a problem based on a finite element space w.r.t. the background mesh restricted to a subdomain, e.g.
 * a fictitious domain method on $\Omega_{-} = \{ \phi < 0 \}$
 * a trace finite element on $\Gamma = \{ \phi = 0\}$
 
In this case you can use the standard finite element space but need to know which degrees of freedom are really used and which ones are unused. We have helper functions for that as well.

As an example we use a marking for a fictitious domain method

In [37]:
VhG = H1(mesh, order=1, dirichlet=[])
freedofs = VhG.FreeDofs()
freedofs &= GetDofsOfElements(VhG,ba_hasneg)

### Marking integrators

Furthermore, the marker fields can be used to restrict integrators to some elements:

In [39]:
integrator = SymbolicBFI(VhG.TrialFunction()*VhG.TestFunction())
integrator.SetDefinedOnElements(ba_hasneg)

## Integration on level set domains